# <center>ThermoPy: Python-based Program for Analyzing and Visualizing Thermochronometric Data</center>

## <center>Samuel Robbins, Chelsea Mackaman-Lofland</center>

## <center> Notebook 1: Helium Outlier Classification and Sample Statistics </center>

## Getting started
You will need to have the following Jupyter packages installed to run ThermoPy:
- pandas
- matplotlib
- seaborn
- numpy
- statsmodels
- scipy
- random
- pathlib
- **xlsxwriter**

Use `pip list` in the cell below to determine which packages you already have:

In [ ]:
pip list

Install any packages that are not already downloaded using `pip install [ package name ]` in the cell below. 

These installations only need to be completed once – users can skip the **Getting started** steps during subsequent ThermoPy uses.

In [ ]:
pip install pathlib

## 0. Import modules
Run this cell to import the necessary Jupyter modules every time you open ThermoPy.

In [ ]:
# ThermoPy Functions
import thermoFuncs as tFunc

import matplotlib
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

--------

## 1. Load data



In these two steps, data are loaded from your Excel template, located using the user-specified `filepath`. (The default filepath assumes your Excel file is located in the same folder as your ThermoPy .ipynb code). The Excel file should have two worksheets:
- a `Samples` worksheet containing sample metadata (sample name, location/elevation information, lithology, etc.).
- an `Aliquots` worksheet containing thermochronometric data for individual aliquots.

ThermoPy links the two worksheets using the Sample ID. All aliquots in the `Aliquots` worksheet should have a sample reference in the `Samples` worksheet.

The outputs of this step include:
- `samples` dataframe - _all data from the `Samples` worksheet_
- `sample_list` - _list of all Sample IDs in the `Samples` worksheet_
- `transect_list` - _list of all specified transects in the `Samples` worksheet_
- `aliquots` dataframe - _all data from the `Aliquots` worksheet_


### Best practice for sample naming schema (SampleIDs)

AHe, AFT, ZHe, and ZFT samples should have chronometer codes built into the sample name. For example, if the sample AN01 has data for all four chronometers, the SampleIDs should be:

- AN01 (apatite He)
- zAN01 (zircon He)
- AN01_AFT (apatite FT)
- AN01_ZFT (zircon FT)

In [ ]:
filepath = 'example_data/ThermoPy_example_data.xlsx'

In [ ]:
samples, sample_list, transect_list, aliquots = tFunc.loadDataExcel(filepath)

Running the code cell below is optional – it simply returns the list of all Sample IDs in the `Samples` worksheet of your Excel upload. We suggest using the Sample IDs list to double-check that your data upload was successful and as a reference once you've moved on to Step 2.

In [ ]:
sample_list

In [ ]:
transect_list

--------

## 2. Plot He aliquot effective uranium (eU) & equivalent spherical radius (Rs or Rft) data

The effective uranium (eU) concentration weights parent isotopes based on their alpha productivity and may serve as a proxy for total radiation damage due to alpha decay in a crystal, as described in Cooperdock et al. (2019) and Flowers et al. (2022a).

The grain size of He aliquots may be represented by the **volume-equivalent spherical radius (Rs)** or the **Ft-equivalent spherical radius (Rft)**, as defined in Ketcham et al. (2011) and Cooperdock et al. (2019). The user-defined `radius` variable allows users to choose the grain size parameter that best suites their dataset.

Finally, note that this step visualizes **apatite and zircon He grain data**. As such, the `Samples` list should not include any fission track (FT) samples!

Inputs for this function include a user-defined `Samples` list, the **`aliquots`** dataframe created in Step 1, and a user-defined `radius` parameter (Rs or Rft). 
```
tFunc.plot_samples_eU_Rft(Samples, aliquots, radius, savefig = True)
```

To plot **ALL He samples** in the database run the below cell.

In [ ]:
## User-defined inputs
radius = 'Rs' # Options are 'Rs' or 'Rft'. Make sure the value is present in your input file!
plot_histogram = True # True or False

# Specify Histogram parameters
bin_width=2
kde_overlay=True

# List of ALL He samples
all_He_samples = [sample_name for (sample_name, sample_type) in sample_list if 'HE' in sample_type.upper()]

tFunc.plot_samples_eU_Rft(all_He_samples, aliquots, radius, plot_histogram, bin_width, kde_overlay, savefig = False) 

To plot **only specific He samples** in the database, add the specified samples to the list in the cell below, and run cell.

In [ ]:
## User-defined inputs
radius = 'Rs' # Options are 'Rs' or 'Rft'. Make sure the value is present in your input file!
plot_histogram = True # True or False

# Specify Histogram parameters
bin_width=3
kde_overlay=True

## To plot specific samples, fill in the below 'Samples' list, uncomment the following 2 lines of code and run cell
He_samples_to_plot = ['AN01','zAN01','AN03','zAN03'] # User-defined list of 'SampleIDs' to plot

tFunc.plot_samples_eU_Rft(He_samples_to_plot, aliquots, radius, plot_histogram, bin_width, kde_overlay, savefig = True) 

--------

## 3. He aliquot outlier calculation and rejection/retention

The steps below identify potential outlier aliquots for each He sample in the `sample_list` (defined in Step 1) using the He grain data in the `aliquots` dataframe.

Outliers may be assessed using the Inter-Quartile Range (IQR) or Chauvenet's Criterion methods of outlier detection. Both methods assume the data are _normally distributed_.

There are no outputs to this step – it only allows the user to **_view_** the default outlier classifcations based on the specified statistical method. 

**No sample stats are calculated in this step.**

### <font color = royalblue> 3a. IQR outlier method

In [ ]:
stat_scheme = 'IQR' # Options: IQR, Chauvenet

tFunc.viewOutliers(samples, aliquots, sample_list, stat_scheme)

### <font color = forestgreen>3b. Chauvenet's criterion 

In [ ]:
stat_scheme = 'Chauvenet' # Options: IQR, Chauvenet

tFunc.viewOutliers(samples, aliquots, sample_list, stat_scheme)

--------

## 4. Calculate sample statistics and export data

The code below calculates central tendency statistics and uncertainties based on the unweighted, weighted by inverse variance, and weighted by squared relative deviation equations in Table 2 of [Flowers et al. (2022b)](https://scholar.google.com/citations?view_op=view_citation&hl=en&user=TZmDk-4AAAAJ&sortby=pubdate&citation_for_view=TZmDk-4AAAAJ:yD5IFk8b50cC).

### Calculate full sample stats

#### <font color = salmon>Ex. I. Summary IQR Statistics w/ Defualt Outliers Rejected</font>

In [ ]:
stat_scheme = 'IQR' # Options : IQR, Chauvenet

keep_all_outliers = False # True or False - will mark ALL aliquot data as 'keep'

aliquots_to_keep = None # None or list of aliquots [] to change from 'reject' to 'keep' – if None, default outliers will be honored
aliquots_to_reject = None # None or list of aliquots [] to change from 'keep' to 'reject'

save_data_excel = True # True or False
filename = 'summary_statistics_IQR.xlsx'

In [ ]:
sample_stats, aliquot_stats = tFunc.calculateFullSummaryStats(samples, aliquots, sample_list, 
                                                        stat_scheme, keep_all_outliers, aliquots_to_keep, aliquots_to_reject,
                                                        save_data_excel, filename)

#### <font color = royalblue>Ex. II. Summary IQR Statistics w/ all Outliers Kept</font>

In [ ]:
stat_scheme = 'IQR' # Options : IQR, Chauvenet

keep_all_outliers = True # True or False - will mark ALL aliquot data as 'keep'

aliquots_to_keep = None # None or list of aliquots [] to change from 'reject' to 'keep' – if None, default outliers will be honored
aliquots_to_reject = None # None or list of aliquots [] to change from 'keep' to 'reject'

save_data_excel = True # True or False
filename = 'summary_statistics_IQR_no_outliers.xlsx'

In [ ]:
sample_stats2, aliquot_stats2 = tFunc.calculateFullSummaryStats(samples, aliquots, sample_list, 
                                                        stat_scheme, keep_all_outliers, aliquots_to_keep, aliquots_to_reject,
                                                        save_data_excel, filename)

#### <font color = seagreen>Ex. III. Summary Chauvenet's Statistics w/ Default Outliers Rejected</font>

In [ ]:
stat_scheme = 'Chauvenet' # Options : IQR, Chauvenet

keep_all_outliers = False # True or False - will mark ALL aliquot data as 'keep'

aliquots_to_keep = None # None or list of aliquots [] to change from 'reject' to 'keep' – if None, default outliers will be honored
aliquots_to_reject = None # None or list of aliquots [] to change from 'keep' to 'reject'

save_data_excel = True # True or False
filename = 'summary_statistics_Chauvenet.xlsx'

In [ ]:
sample_stats3, aliquot_stats3 = tFunc.calculateFullSummaryStats(samples, aliquots, sample_list, 
                                                        stat_scheme, keep_all_outliers, aliquots_to_keep, aliquots_to_reject,
                                                        save_data_excel, filename)

#### <font color = orange>Ex. IV. Summary Chauvenet's Statistics w/ all Outliers Kept</font>

In [ ]:
stat_scheme = 'Chauvenet' # Options : IQR, Chauvenet

keep_all_outliers = True # True or False - will mark ALL aliquot data as 'keep'

aliquots_to_keep = None # None or list of aliquots [] to change from 'reject' to 'keep' – if None, default outliers will be honored
aliquots_to_reject = None # None or list of aliquots [] to change from 'keep' to 'reject'

save_data_excel = True # True or False
filename = 'summary_statistics_Chauvenet_no_outliers.xlsx'

In [ ]:
sample_stats4, aliquot_stats4 = tFunc.calculateFullSummaryStats(samples, aliquots, sample_list, 
                                                        stat_scheme, keep_all_outliers, aliquots_to_keep, aliquots_to_reject,
                                                        save_data_excel, filename)

-------------

## End of Notebook 1
### Notebook 1 output (which serves as input to Notebook 2...)

The output is an Excel sheet with sample statistics and outlier information, to be loaded into the second notebook in the ThermoPy series.